# Setting

In [ ]:
# -*- coding: utf-8 -*-
import pickle
import sys
import os
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

path = 'your_path'
os.chdir(path)

In [ ]:
!pip install opencc

# Get 萌典 moe-dict

- https://github.com/g0v/moedict-data.git
- https://github.com/g0v/moedict-epub.git

In [ ]:
!sudo apt update
!sudo apt install -y python g++ make nodejs python-lxml curl npm

In [ ]:
!npm i
!pip install lxml
!sudo npm i -g gulp

In [ ]:
!git clone --depth 1 https://github.com/g0v/moedict-data.git
!git clone --depth 1 https://github.com/g0v/moedict-epub.git
!cp -v moedict-data/dict-revised.json moedict-epub/
!cd moedict-epub

# Build my DataFrame

沒有採用的資料："non_radical_stroke_count", "radical", "stroke_count"

In [ ]:
import json
import pandas as pd

# read json
with open(path+'/moedict-data/dict-revised.json', "r") as read_file:
    data = json.load(read_file)
print(type(data),len(data))

<class 'list'> 163087


In [ ]:
import pandas as pd

# function to extract information from each dictionary and create a list of dictionaries
def extract_info(d):
    title = d['title']
    rows = []
    for heteronym in d['heteronyms']:
    # INFO in the 2nd layer 
        if 'bopomofo' in heteronym:
            bopomofo = heteronym['bopomofo']
        else:
            bopomofo = ''
        if 'pinyin' in heteronym:
            pinyin = heteronym['pinyin']
        else:
            pinyin = ''
            
        for i, definition in enumerate(heteronym['definitions']):
        # INFO in the 3rd layer (iterrating each senses of this lemma)
            lemma_text = title
            sense_idx = i+1
            definition_text = definition['def']
            
            pos = definition.get('type', '')

            if 'quote' in definition:
                quote = definition['quote'][0]
            else:
                quote = ''
            if 'example' in definition:
                examples = [s[s.index('「')+1:s.index('」')] for s in definition['example']]
            else:
                examples = ''
            
            if 'anotonyms' in definition:
                anotonyms = definition['anotonyms'].split(',')
            else:
                anotonyms = []
            if 'synonyms' in definition:
                synonyms = definition['synonyms'].split(',')
            else:
                synonyms = []      

            row_dict = {'lemma_text': lemma_text,
                        'sense_idx': sense_idx,
                        'definition': definition_text,
                        'synonyms': synonyms,
                        'anotonyms': anotonyms,
                        'quote': quote,
                        'examples': examples,
                        'POS': pos,
                        'bopomofo': bopomofo,
                        'pinyin': pinyin}
            rows.append(row_dict)
    return rows

In [ ]:
# Extract information from the list of dictionaries
rows = []
for d in data:
    rows += extract_info(d)

# Create a pandas DataFrame from the list of dictionaries
df = pd.DataFrame(rows, columns=['lemma_text','sense_idx','definition', 'synonyms', 'anotonyms', 
                                    'quote', 'examples','POS','bopomofo','pinyin'])

print(len(df))

In [ ]:
# combine quotes & examples into []
# only examples -> [examples], only quotes -> [quote]

def merge_quotes_and_examples(row):
    quote = row['quote']
    examples = row['examples']
    if not quote and not examples:
        return []
    elif quote and examples:
        if type(quote) == str:
            quote = [quote]
        result = []
        result+=examples
        result+=quote
        return result
    elif examples:
        return examples
    else:
        if type(quote) == str:
            quote = [quote]
            return quote

# apply the function to each row of the DataFrame
df['examples_quote'] = df.apply(merge_quotes_and_examples, axis=1)

# drop the original quote and examples columns
df.drop(columns=['quote', 'examples'], inplace=True)

print(len(df))

In [ ]:
'''
optional
'''

# find rows whose lemma_text contains {} \d a-zA-Z
index = df[df['lemma_text'].str.contains(r'[\{\}\da-zA-Z]')].index
len(index)

# drop the rows by their idxs
df.drop(index, inplace=True)
print(len(df))

In [ ]:
 # save to csv
df.to_csv('moe_flat.csv', encoding='utf-8',index=False)